In [15]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Backtest, StopLoss
from pybacktestchain.blockchain import load_blockchain
from datetime import datetime
import polars as pl
import plotly.express as px

import get_all_tickers.get_tickers as gt

In [17]:
gt.SectorConstants.TECH

'Technology'

In [22]:
gt.get_tickers(AMEX=False, NASDAQ=False)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 8, saw 2


In [23]:
import requests

headers = {
    'authority': 'api.nasdaq.com',
    'accept': 'application/json, text/plain, */*',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'origin': 'https://www.nasdaq.com',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.nasdaq.com/',
    'accept-language': 'en-US,en;q=0.9',
}

params = (
    ('tableonly', 'true'),
    ('limit', '25'),
    ('offset', '0'),
    ('download', 'true'),
)

r = requests.get('https://api.nasdaq.com/api/screener/stocks', headers=headers, params=params)
data = r.json()['data']
df = pl.DataFrame(data['rows'])

In [33]:
df.filter(~pl.col('marketCap').eq(""), ~pl.col('ipoyear').eq("")).with_columns(pl.col('volume').cast(pl.Int128), pl.col('marketCap').cast(pl.Float64))

symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
str,str,str,str,str,i128,f64,str,str,str,str,str
"""A""","""Agilent Technologies Inc. Comm…","""$137.47""","""0.47""","""0.343%""",1369873,3.9261e10,"""United States""","""1999""","""Biotechnology: Laboratory Anal…","""Industrials""","""/market-activity/stocks/a"""
"""AA""","""Alcoa Corporation Common Stock…","""$35.91""","""-0.09""","""-0.25%""",2942573,9.2775e9,"""United States""","""2016""","""Aluminum""","""Industrials""","""/market-activity/stocks/aa"""
"""AACG""","""ATA Creativity Global American…","""$0.9462""","""-0.0329""","""-3.36%""",20957,3.0278523e7,"""China""","""2008""","""Other Consumer Services""","""Real Estate""","""/market-activity/stocks/aacg"""
"""AACT""","""Ares Acquisition Corporation I…","""$10.98""","""0.00""","""0.00%""",14192,0.0,"""""","""2023""","""Blank Checks""","""Finance""","""/market-activity/stocks/aact"""
"""AAM""","""AA Mission Acquisition Corp. C…","""$10.11""","""-0.01""","""-0.099%""",619,0.0,"""""","""2024""","""""","""""","""/market-activity/stocks/aam"""
…,…,…,…,…,…,…,…,…,…,…,…
"""ZUO""","""Zuora Inc. Class A Common Stoc…","""$9.96""","""0.01""","""0.101%""",3282202,1.5309e9,"""United States""","""2018""","""Computer Software: Prepackaged…","""Technology""","""/market-activity/stocks/zuo"""
"""ZVIA""","""Zevia PBC Class A Common Stock""","""$4.42""","""0.18""","""4.245%""",905385,3.22722238e8,"""United States""","""2021""","""Beverages (Production/Distribu…","""Consumer Staples""","""/market-activity/stocks/zvia"""
"""ZVSA""","""ZyVersa Therapeutics Inc. Comm…","""$1.47""","""0.29""","""24.576%""",38269055,3.445961e6,"""United States""","""2022""","""Biotechnology: Pharmaceutical …","""Health Care""","""/market-activity/stocks/zvsa"""


In [14]:
# Set verbosity for logging
verbose = False  # Set to True to enable logging, or False to suppress it

backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    name_blockchain='backtest',
    verbose=verbose
)

backtest.run_backtest()

block_chain = load_blockchain('backtest')


INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\faune\miniconda3\envs\pybacktestchain-gui\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning:

Degrees of freedom <= 0 for slice

c:\Users\faune\miniconda3\envs\pybacktestchain-gui\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning:

divide by zero encountered in divide

c:\Users\faune\miniconda3\envs\pybacktestchain-gui\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning:

invalid value encountered in multiply

INFO:root:Stop loss triggered for NVDA at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for NFLX at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-03 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:root:Rebalanci

In [3]:
block_chain.chain[2].name_backtest

'VioletJaguarSoldier'

In [12]:
test = (
    pl.read_csv(
        source=fr".\backtests\{block_chain.chain[4].name_backtest}.csv"
    )
    .select(
        pl.col('Date').str.to_date(), 
        pl.exclude('', 'Date')
    )
    .with_columns(dollar_amount = pl.col('Quantity').mul(pl.col('Price')))
)

In [13]:
px.line(test.select('Cash'))